In [7]:
import os
import copy
import random
import math

In [8]:
class City:
    
    def __init__(self, city):
        self.number = city[0]
        self.x = city[1]
        self.y = city[2]
        
    def __repr__(self):
        return f'({self.x},{self.y})'
        
    def __str__(self):
        return f'City No:{self.number}, X:{self.x}, Y:{self.y}'


class TSP:
    def __init__(self, path) -> None:
        self.cities = []
        self.city_dict = {}
        self.opt_tour = []
        self.init_tsp(path)
        
    def init_tsp(self, path):
        
        # Populate initial configuration of cities
        file = open(f'tsp_files/vlsi/{path}/{path}.tsp').readlines()
        
        self.name = file[0]
        self.size = int(file[5].split(" ")[-1].replace('\n', ''))
        
        for city_line in file[8:-1]:
            city = city_line.split(" ")
            city[1] = int(city[1])
            city[2] = int(city[2].replace('\n', ''))
            
            self.cities.append(City(city))
            
            self.city_dict[city[0]] = City(city)
            
        self.cities.append(self.cities[0])
        
        
        # Store data on optimal path
        file = open(f'tsp_files/vlsi/{path}/{path}.tour').readlines()
        
        self.opt_tour_length = int(file[1].split(" ")[-1].replace("\n", ""))
        
        for city_line in file[5:-2]:
            city_num = city_line.replace("\n", "")
            self.opt_tour.append(self.city_dict[city_num])
        
    def set_cities(self, cities):
        self.cities = cities
        
    def get_cities(self):
        return copy.deepcopy(self.cities)
    
    def cities_to_numbers(self, cities):
        
        return [city.number for city in cities]
    
    def numbers_to_cities(self, city_num_list):
        return [self.city_dict[num] for num in city_num_list]

In [75]:
class Chromosome:
    
    def __init__(self, cities):
        
        self.cities = cities
        self.fitness = None
        
    def __repr__(self):
        return f"(Fitness: {self.fitness}, {self.cities[:2]} ... {self.cities[-2:]})"
    
    def __str__(self):
        return f"Fitness: {self.fitness}\n{self.cities}"
    
    def set_fitness(self, fitness):
        self.fitness = fitness

class GeneticAlgorithm:
    
    def __init__(self, pop_size, cross_prop, tsp):
        self.POP_SIZE = pop_size
        self.cross_prop = cross_prop
        self.population = []
        self.tsp = tsp
        
        self.init_population(tsp)
        
    def init_population(self, tsp):
        
        for i in range(self.POP_SIZE):
            temp_tsp = copy.deepcopy(tsp)
            temp_city_list = temp_tsp.get_cities()
            start_city = temp_city_list[0]
            temp_city_list = temp_city_list[1:-1]
            
            random.shuffle(temp_city_list)
            temp_city_list.insert(0, start_city)
            temp_city_list.insert(len(temp_city_list), start_city)
            
#             temp_tsp.set_cities(temp_city_list)

            chrm = Chromosome(temp_city_list)
            
            self.population.append(chrm)
            
    def eval_fitness(self, indv):
        
        total_fitness = 0
        
        for i in range(len(indv.cities)-1):
            total_fitness += math.dist([indv.cities[i].x, indv.cities[i].y], [indv.cities[i+1].x, indv.cities[i+1].y])
            
        return round(total_fitness)
    
    def roulette_selection(self, pop):
        total_fitness = sum([1/x.fitness for x in pop])
        prob_list = [round((1/x.fitness)/total_fitness, 2) for x in pop]
        
        rw_rand_num = round(random.uniform(0.0, sum(prob_list)), 2)

        curr_wheel_fitness = 0

        for i, indv_prob in enumerate(prob_list):
            curr_wheel_fitness += indv_prob

            if rw_rand_num < curr_wheel_fitness:
                return i
            
    def pmx_crossover(self, p1, p2):
        
        parent_length = len(p1)
        left_bound = random.randint(0, parent_length-1)
        right_bound = random.randint(left_bound, parent_length-1)
        p1_map = []
        p2_map = []
        
        c1 = [None for x in range(parent_length)]
        c2 = [None for x in range(parent_length)]
        
        # Swap genes in parents 
        for i in range(left_bound, right_bound+1):
            p1_map.append(p2[i])
            p2_map.append(p1[i])
            c1[i] = p2[i]
            c2[i] = p1[i]
            
        # Fill values before left bound of children
        for i in range(left_bound):
            valid_c1 = False
            valid_c2 = False
            temp_c1_city = p1[i]
            temp_c2_city = p2[i]
            
            while valid_c1 == False:
                if temp_c1_city in p1_map:
                    temp_c1_city = p2_map[p1_map.index(temp_c1_city)]
                else:
                    valid_c1 = True
            
            c1[i] = temp_c1_city
            
            while valid_c2 == False:
                if temp_c2_city in p2_map:
                    temp_c2_city = p1_map[p2_map.index(temp_c2_city)]
                else:
                    valid_c2 = True
            
            c2[i] = temp_c2_city
            
        # Fill values after right bound of children
        for i in range(right_bound+1, parent_length):
            valid_c1 = False
            valid_c2 = False
            temp_c1_city = p1[i]
            temp_c2_city = p2[i]
            
            while valid_c1 == False:
                if temp_c1_city in p1_map:
                    temp_c1_city = p2_map[p1_map.index(temp_c1_city)]
                else:
                    valid_c1 = True
            
            c1[i] = temp_c1_city
            
            while valid_c2 == False:
                if temp_c2_city in p2_map:
                    temp_c2_city = p1_map[p2_map.index(temp_c2_city)]
                else:
                    valid_c2 = True
            
            c2[i] = temp_c2_city
            
        return (c1, c2)
    
    def book_selection(self, pop, sel_size):
        # Book's implementation of selection
        
        sel_pop = []
        
        while len(sel_pop) < sel_size:
            sel_idx = self.roulette_selection(pop)
            sel_pop.append(pop.pop(sel_idx))
                    
        return sel_pop
    
    def book_crossover(self, pop, cross_size, tsp):
        # Book's implementation of crossover
        # Two Point Crossover
        
        cross_pop = []
        
        while len(cross_pop) < cross_size:
            p1_idx = self.roulette_selection(pop)
            p2_idx = self.roulette_selection(pop)
            while p1_idx == p2_idx:
                p2_idx = self.roulette_selection(pop)

            start_point = pop[0].cities[0]
            parent1 = self.tsp.cities_to_numbers(pop[p1_idx].cities[1:-1])
            parent2 = self.tsp.cities_to_numbers(pop[p2_idx].cities[1:-1])

    #         print(start_point)
    #         print(parent1)
    #         print(parent2)

            child_1, child_2 = self.pmx_crossover(parent1, parent2)

            child_1 = tsp.numbers_to_cities(child_1)
            child_2 = tsp.numbers_to_cities(child_2)

            child_1.insert(0, start_point)
            child_1.append(start_point)

            child_2.insert(0, start_point)
            child_2.append(start_point)
            
            cross_pop.append(Chromosome(child_1))
            cross_pop.append(Chromosome(child_2))
            
        return cross_pop
        
    def fit(self):
        
        for indv in self.population:
            indv.set_fitness(self.eval_fitness(indv))
            
        self.population.sort(key= lambda x: x.fitness)
        
        temp_pop = copy.deepcopy(self.population)
        # Definies the amount of chromosomes that will be selected. the last term ensures the size of the crossover population is a multiple of 2 as crossover produces pairs.
        sel_size = self.POP_SIZE - (round(self.cross_prop*self.POP_SIZE) + round(self.cross_prop*self.POP_SIZE)%2)
        
        sel_pop = self.book_selection(temp_pop, sel_size)
        
        temp_pop = copy.deepcopy(self.population)
        cross_size = (round(self.cross_prop*self.POP_SIZE) + round(self.cross_prop*self.POP_SIZE)%2)
        cross_pop = self.book_crossover(temp_pop, cross_size, self.tsp)
        
        for indv in cross_pop:
            indv.set_fitness(self.eval_fitness(indv))
            
        cross_pop.sort(key=lambda x: x.fitness)
        
        print(self.population)
        print()
        print(cross_pop)

In [76]:
path = "xqf131"
t1 = TSP(path)

ga = GeneticAlgorithm(10, 0.5, t1)

In [92]:
ga.fit()

[(Fitness: 4434, [(0,13), (28,34)] ... [(57,44), (0,13)]), (Fitness: 4524, [(0,13), (5,37)] ... [(5,25), (0,13)]), (Fitness: 4542, [(0,13), (74,12)] ... [(33,15), (0,13)]), (Fitness: 4652, [(0,13), (28,47)] ... [(48,22), (0,13)]), (Fitness: 4706, [(0,13), (15,25)] ... [(48,22), (0,13)]), (Fitness: 4733, [(0,13), (79,37)] ... [(71,11), (0,13)]), (Fitness: 4756, [(0,13), (2,0)] ... [(57,25), (0,13)]), (Fitness: 4798, [(0,13), (41,23)] ... [(57,12), (0,13)]), (Fitness: 4907, [(0,13), (57,44)] ... [(35,31), (0,13)]), (Fitness: 5021, [(0,13), (34,15)] ... [(25,23), (0,13)])]

[(Fitness: 4286, [(0,13), (28,34)] ... [(57,44), (0,13)]), (Fitness: 4555, [(0,13), (28,47)] ... [(48,22), (0,13)]), (Fitness: 4599, [(0,13), (2,0)] ... [(57,25), (0,13)]), (Fitness: 4679, [(0,13), (79,37)] ... [(71,11), (0,13)]), (Fitness: 4970, [(0,13), (34,15)] ... [(25,23), (0,13)]), (Fitness: 5007, [(0,13), (57,44)] ... [(35,31), (0,13)])]
